# test

In [1]:
# test

In [2]:
import pandas as pd
import petl as etl

In [5]:
reveal = pd.read_csv('data/refined/branches_established_2010.csv')

reveal = reveal.set_index(['region', 'county', 'year'])

month  day  hour                    product  \
region        county    year                                                
East Midlands Bassetlaw 2010      1    9     0            chocolate twist   
                        2010      1    9     0          plastic sunshades   
                        2010      1    9     0         lg washing machine   
                        2010      1    9     0        fresh whole chicken   
                        2010      1    9     0  free range large eggs x12   
...                             ...  ...   ...                        ...   
Wales         Barry     2020     12    2    23          hp dv 3000 laptop   
                        2020     12    2    23                linen shirt   
                        2020     12    2    23   21 day mature rump steak   
                        2020     12    2    23           wellington boots   
                        2020     12    2    23                Almond milk   

                              total_quantity_purchased  amount_in_gbp  \
region        county    year                                            
East Midlands Bassetlaw 2010                     182.0          145.6   
                        2010                      98.0          313.6   
                        2010                     151.0        12080.0   
                        2010                     230.0          257.6   
                        2010                     150.0          180.0   
...                                                ...            ...   
Wales         Barry     2020                     200.0       280000.0   
                        2020                     213.0        26199.0   
                        2020                     214.0          684.8   
                        2020                      99.0         2376.0   
                        2020                     137.0          260.3   

                             product_category  
region        county    year                   
East Midlands Bassetlaw 2010           bakery  
                        2010         clothing  
                        2010       appliances  
                        2010      meat & fish  
                        2010            dairy  
...                                       ...  
Wales         Barry     2020        computing  
                        2020         clothing  
                        2020      meat & fish  
                        2020         clothing  
                        2020            dairy  

[39968067 rows x 7 columns]

In [9]:
reveal.count()

region                      39968067
county                      39968067
year                        39968067
month                       39968067
day                         39968067
hour                        39968067
product                     39968067
total_quantity_purchased    39964122
amount_in_gbp               39968067
product_category            39002057
dtype: int64

In [4]:
test_df = pd.read_csv('data/raw/2010-2020_Bassetlaw_outlet.csv')
test_df_1 = pd.read_csv('data/raw/2010-2020_Castle_Point_outlet.csv')


test_df_1 = test_df_1.set_index(['year', 'sku'])

test_df_1

test_df = test_df.set_index(['year', 'sku'])

test_df.append(test_df_1)

test_df

month  day  hour quantity_purchased  \
year sku                                                              
2010 chocolate twist                1    9     0                182   
     plastic sunshades              1    9     0                 98   
     lg washing machine             1    9     0                151   
     fresh whole chicken            1    9     0                230   
     free range large eggs x12      1    9     0                150   
...                               ...  ...   ...                ...   
2020 chicken sausages              12   23    23                225   
     silk scarf                    12   23    23                  8   
     denim jeams                   12   23    23                185   
     cod fillets                   12   23    23                 48   
     organic raspberry yoghurt     12   23    23                103   

                                amount_in_gbp  
year sku                                       
2010 chocolate twist                    145.6  
     plastic sunshades                  313.6  
     lg washing machine               12080.0  
     fresh whole chicken                257.6  
     free range large eggs x12          180.0  
...                                       ...  
2020 chicken sausages                   450.0  
     silk scarf                        1072.0  
     denim jeams                       5550.0  
     cod fillets                         96.0  
     organic raspberry yoghurt          144.2  

[1119662 rows x 5 columns]